## ПАРАМЕТРЫ

In [19]:
# Порог
tres =250
# Вид фильтрации  0-нет фильтрации, 1 - среднее, 2 - медиана
filtr_type = 2
# Окно фильтрации
ww = 5
# шаг diff
step = 5
# Удалять нулевые столбцы? 0 - не удалять
drop = 1

In [20]:
import pandas as pd
import numpy as np
from statistics import median
from statistics import mean

In [21]:
X_train = np.load ( 'X_train.npy')
X_train.shape

(165, 40, 60)

In [22]:
X_test = np.load ( 'X_test.npy')
X_test.shape

(166, 40, 60)

In [23]:
y_train = pd.read_csv('y_train.csv')

In [24]:
# Пороговая фильтрация train
X_tres_train = np.copy(X_train)

for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        for k in range(X_train.shape[2]):
            if X_train[i][j][k] <= tres:
                X_tres_train[i][j][k] = 0
            else:
                X_tres_train[i][j][k] = X_train[i][j][k]
            

In [25]:
# Пороговая фильтрация test
X_tres_test = np.copy(X_test)

for i in range(X_test.shape[0]):
    for j in range(X_test.shape[1]):
        for k in range(X_test.shape[2]):
            if X_test[i][j][k] <= tres:
                X_tres_test[i][j][k] = 0
            else:
                X_tres_test[i][j][k] = X_test[i][j][k]              
            

In [26]:
# Фильтрация train
# Вид фильтрации  0-нет фильтрации, 1 - среднее, 2 - медиана
X_m_train = np.copy(X_tres_train)

if filtr_type == 1:
    for i in range(X_train.shape[0]):
        for j in range(X_train.shape[1]):
            for k in range(X_train.shape[2]):
                X_m_train[i][j][k] = mean(list(X_tres_train[i][j])[k:(k+ww)])
elif filtr_type == 2:
    for i in range(X_train.shape[0]):
        for j in range(X_train.shape[1]):
            for k in range(X_train.shape[2]):
                X_m_train[i][j][k] = median(list(X_tres_train[i][j])[k:(k+ww)])   

In [27]:
# Фильтрация test
# Вид фильтрации  0-нет фильтрации, 1 - среднее, 2 - медиана
X_m_test = np.copy(X_tres_test)

if filtr_type == 1:
    for i in range(X_test.shape[0]):
        for j in range(X_test.shape[1]):
            for k in range(X_test.shape[2]):
                X_m_test[i][j][k] = mean(list(X_tres_test[i][j])[k:(k+ww)])
elif filtr_type == 2:
    for i in range(X_train.shape[0]):
        for j in range(X_train.shape[1]):
            for k in range(X_train.shape[2]):
                X_m_test[i][j][k] = median(list(X_tres_test[i][j])[k:(k+ww)])

In [28]:
# Формирование признаков train
n_features = 8              # количество признаков
ind_feat = n_features - 1    # индекс признака (для удобства индексации)
X_d_train = np.zeros((165, 40, n_features))
for i in range(165):
    chan_df =  pd.DataFrame(X_m_train[i]).T

    for j in range(40):
        diff_current = chan_df[j].diff(step)
        
        X_d_train[i][j][ind_feat] = chan_df[j].mean()
        X_d_train[i][j][ind_feat-1] = chan_df[j].median()
        X_d_train[i][j][ind_feat-2] = chan_df[j].std()
        X_d_train[i][j][ind_feat-3] = chan_df[j][0]
        X_d_train[i][j][ind_feat-4] = chan_df[j][59]
        X_d_train[i][j][ind_feat-5] = diff_current.max() - diff_current.min()
        X_d_train[i][j][ind_feat-6] = diff_current.idxmax()
        X_d_train[i][j][ind_feat-7] = diff_current.idxmin()
X_d_train.shape

(165, 40, 8)

In [29]:
# Формирование признаков test
X_d_test = np.zeros((166, 40, n_features))
for i in range(166):
    chan_df_ =  pd.DataFrame(X_m_test[i]).T

    for j in range(40):
        diff_current_ = chan_df_[j].diff(step)
        
        X_d_test[i][j][ind_feat] = chan_df_[j].mean()
        X_d_test[i][j][ind_feat-1] = chan_df_[j].median()
        X_d_test[i][j][ind_feat-2] = chan_df_[j].std()
        X_d_test[i][j][ind_feat-3] = chan_df_[j][0]
        X_d_test[i][j][ind_feat-4] = chan_df_[j][59]
        X_d_test[i][j][ind_feat-5] = diff_current_.max() - diff_current_.min()
        X_d_test[i][j][ind_feat-6] = diff_current_.idxmax() 
        X_d_test[i][j][ind_feat-7] = diff_current_.idxmin() 
X_d_test.shape

(166, 40, 8)

In [30]:
dim2 = X_d_train.shape[1]*X_d_train.shape[2]
X_train_2d_ = X_d_train.reshape((X_d_train.shape[0], dim2))
X_test_2d_ = X_d_test.reshape((X_d_test.shape[0],dim2))

In [31]:
X_train_2d = pd.DataFrame(X_train_2d_)
X_test_2d = pd.DataFrame(X_test_2d_)

In [32]:
low_sig_list_train = []
for col in X_train_2d.columns:
    if X_train_2d[col].max()<= 0:
        low_sig_list_train.append(col)

len(low_sig_list_train)

144

In [33]:
low_sig_list_test = []
for col in X_test_2d.columns:
    if X_test_2d[col].max()<= 0:
        low_sig_list_test.append(col)

len(low_sig_list_test)

144

Если len(low_sig_list_train) не равна len(low_sig_list_test) модель тестовую выборку не обрабатывает. Тогда надо подобрать порог tres.

In [34]:
if drop != 0:
    X_train_2d.drop(low_sig_list_train, axis=1, inplace=True)
    X_test_2d.drop(low_sig_list_test, axis=1, inplace=True)

In [35]:
y_train = y_train['Class']

In [36]:
from sklearn.model_selection import train_test_split
# выделим 20% данных на тестовую выборку (параметр test_size)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_train_2d, y_train, test_size=0.2, random_state=0, stratify=y_train)

In [37]:
from sklearn.linear_model import LogisticRegression
#Обучаем модель
logreg_0 = LogisticRegression(random_state=42, max_iter = 1000)
logreg_0.fit(X_train_1, y_train_1)

LogisticRegression(max_iter=1000, random_state=42)

In [38]:
y_pred = logreg_0.predict(X_test_1)

In [39]:
from sklearn.metrics import f1_score
f1_score(y_test_1, y_pred, average='weighted')

1.0

In [40]:
y_pred_test = logreg_0.predict(X_test_2d)

In [41]:
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission
submission['Class'] = y_pred_test
submission.to_csv('submission.csv', index=False)

In [42]:
submission

,Id,Class
0,165,10
1,166,1
2,167,10
3,168,13
4,169,4
...,...,...
161,326,14
162,327,13
163,328,3
164,329,9
